In [1]:
from typing import Dict, List, Optional, Tuple
import dedupe
import pandas as pd
import json
import logging

In [2]:
def process_zip_for_loans_df_gov(zip):
    try:
        zip_str = str(int(zip))
        return zip_str
    except Exception as _:
        return None



In [3]:
matched_df_1=pd.read_csv("data/output/matched_df_1.csv")
matched_df_2=pd.read_csv("data/output/matched_df_2.csv")

In [4]:
matched_df_1["zip"]=matched_df_1["zip"].apply(process_zip_for_loans_df_gov)

In [5]:
def process_state_clean(state):
    if isinstance(state,str) and state.lower()=="colorado":
        return "CO"
    else:
        return state 

In [6]:
matched_df_1["state_clean"]=matched_df_1["state_clean"].apply(process_state_clean)

In [8]:
matched_df_2["zip"]

0      80204.0
1      80229.0
2      80205.0
3      80216.0
4      80211.0
        ...   
108    80216.0
109    80223.0
110    80216.0
111    80214.0
112    80002.0
Name: zip, Length: 113, dtype: float64

In [9]:
matched_df_2["zip"]=matched_df_2["zip"].apply(process_zip_for_loans_df_gov)

In [10]:
matched_df_2["zip"]

0      80204
1      80229
2      80205
3      80216
4      80211
       ...  
108    80216
109    80223
110    80216
111    80214
112    80002
Name: zip, Length: 113, dtype: object

In [11]:
matched_data_1=matched_df_1.to_dict(orient="index")
matched_data_2=matched_df_2.to_dict(orient="index")

In [12]:
type(matched_data_1)

dict

In [13]:
fields_for_matching=[dedupe.variables.String("business_name"),
                         dedupe.variables.Exact("zip"),
                         dedupe.variables.String("address_clean"),
                         dedupe.variables.String("city_clean"),
                         dedupe.variables.String("state_clean")]

In [14]:
matches=[]
for k,v in matched_data_1.items():
    matches.append([v,matched_data_2[k]])

In [15]:
training_pairs={
    "match":[{"__class__":"tuple","__value__":item} for item in matches],
    "distinct":[]
}

In [16]:
with open("data/input/training_file.json","w") as fp:
    fp.write(json.dumps(training_pairs,indent=2))

In [17]:
content=None
with open("data/extra/training_file.json","r") as fp:
    content=json.loads(fp.read())

In [18]:
matches=content["match"]

In [19]:
business_name_tuple=[(item["__value__"][0].get("business_name",None),item["__value__"][1].get("business_name",None)) for item in matches]

In [20]:
import csv

In [21]:
with open("data/extra/business_name.csv",mode="w",newline="") as fp:
    fieldnames=["name_1","name_2"]
    writer=csv.DictWriter(fp, fieldnames=fieldnames)
    writer.writeheader()
    for name_1,name_2 in business_name_tuple:
        output_row={"name_1":name_1,"name_2":name_2}
        writer.writerow(output_row)

In [22]:
matched_df_2.dtypes

Unnamed: 0          int64
loan_amount       float64
business_name      object
address            object
city               object
state              object
zip                object
naics_code        float64
business_type      object
race_ethnicity     object
gender             object
veteran            object
non_profit           bool
jobs_reported     float64
date_approved      object
lender             object
cd                 object
source_file        object
na_flag              bool
dupe_flag            bool
year_approved       int64
address_clean      object
state_clean        object
city_clean         object
dtype: object